<a href="https://colab.research.google.com/github/Rahul-G06/proving-gr-using-ligo/blob/main/notebooks/obtain_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Repo:
https://github.com/Rahul-G06/proving-gr-using-ligo.git


In [1]:
! pip install -q 'gwosc' 'gwpy'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 45.0.6 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 45.0.6 which is incompatible.


In [2]:
# Check the version of the package gwosc you are using
import gwosc
print(gwosc.__version__)

0.8.1


In [5]:
from gwosc.datasets import event_gps
from gwpy.timeseries import TimeSeries

event = 'GW150914'
gps = event_gps(event)
print("GPS:", gps)

segment = (gps - 8, gps + 8)

ldata = TimeSeries.fetch_open_data('L1', *segment, cache=True, sample_rate=16384)
print("L1 data")
print(ldata)

hdata = TimeSeries.fetch_open_data('H1', *segment, cache=True, sample_rate=16384)
print("H1 data")
print(hdata)

GPS: 1126259462.4
L1 data
TimeSeries([-1.04988177e-18, -1.03213623e-18, -1.05331045e-18,
            ..., -1.09339536e-18, -1.07342380e-18,
            -1.08303029e-18]
           unit: dimensionless,
           t0: 1126259454.3999634 s,
           dt: 6.103515625e-05 s,
           name: Strain,
           channel: None)
H1 data
TimeSeries([-2.30941416e-19, -2.45758980e-19, -2.37722078e-19,
            ..., -7.48230568e-21, -3.84636665e-20,
            -2.90201667e-20]
           unit: dimensionless,
           t0: 1126259454.3999634 s,
           dt: 6.103515625e-05 s,
           name: Strain,
           channel: None)


In [7]:
ldata.write("/content/proving-gr-using-ligo/data/raw/L1_gw150914_analysis.hdf5", format='hdf5')
hdata.write("/content/proving-gr-using-ligo/data/raw/H1_gw150914_analysis.hdf5", format='hdf5')

<Closed HDF5 dataset>

In [8]:
import h5py

psd_duration = 1024
psd_start = gps - 2000
psd_end   = psd_start + psd_duration

strain_H1 = TimeSeries.fetch_open_data('H1', psd_start, psd_end, cache=True)
strain_L1 = TimeSeries.fetch_open_data('L1', psd_start, psd_end, cache=True)

with h5py.File("/content/proving-gr-using-ligo/data/raw/H1_noise_psd_windows/psd_segment_1024s.hdf5", "w") as f:
    f.create_dataset("strain", data=strain_H1.value)
    f.attrs["detector"] = "H1"
    f.attrs["gps_start"] = psd_start
    f.attrs["gps_end"] = psd_end
    f.attrs["duration"] = psd_duration

with h5py.File("/content/proving-gr-using-ligo/data/raw/L1_noise_psd_windows/psd_segment_1024s.hdf5", "w") as f:
    f.create_dataset("strain", data=strain_L1.value)
    f.attrs["detector"] = "L1"
    f.attrs["gps_start"] = psd_start
    f.attrs["gps_end"] = psd_end
    f.attrs["duration"] = psd_duration